In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# Load the sunspots dataset from a CSV file
data = pd.read_csv('../kaggle_datasets/sunspots.csv',index_col='Date',parse_dates= ['Date']).drop('Unnamed: 0',axis =1)
data.head()

,Monthly Mean Total Sunspot Number
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7


In [39]:
data.rename(columns={'Monthly Mean Total Sunspot Number':'Mean'},inplace=True)

In [45]:
data.head()

,Mean
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7


In [46]:

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

# Fit the ARIMA model
order = (4, 1, 0) 
seasonal_order = (1, 1, 0, 24)
model = SARIMAX(endog=train_data, order=order, seasonal_order=seasonal_order)
results = model.fit()

print(results.summary())



/home/lukmanaliyu/miniconda3/envs/arewads/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/home/lukmanaliyu/miniconda3/envs/arewads/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.86011D+00    |proj g|=  4.24046D-02


 This problem is unconstrained.



At iterate    5    f=  4.85724D+00    |proj g|=  1.33348D-02

At iterate   10    f=  4.84103D+00    |proj g|=  2.82380D-02

At iterate   15    f=  4.83998D+00    |proj g|=  3.00178D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     15     18      1     0     0   3.002D-06   4.840D+00
  F =   4.8399760902586788     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
                                     SARIMAX Results                                      
Dep. Variable:                               Mean   No. Observations:                 2612
Model:             SARIMAX(4, 1, 0)x(1, 1, 0, 24)   Log

In [42]:
# Make predictions on the test data
start_idx = len(train_data)
end_idx = len(train_data) + len(test_data) - 1
predictions = pd.DataFrame(results.get_forecast(steps=len(test_data)).predicted_mean, index=test_data.index)


In [48]:
# Filter out NaN values
valid_predictions = predictions[~np.isnan(test_data['Mean'])]
valid_test_data = test_data[~np.isnan(test_data['Mean'])]


In [50]:
# Evaluate the model
mse = mean_squared_error(valid_test_data, valid_predictions)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

mape = np.mean(np.abs((valid_test_data - valid_predictions) / valid_test_data)) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)


Root Mean Squared Error (RMSE): 488.80400586370763
Mean Absolute Percentage Error (MAPE): Mean             NaN
predicted_mean   NaN
dtype: float64


/home/lukmanaliyu/miniconda3/envs/arewads/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [47]:
test_data.columns

Index(['Mean'], dtype='object')